# SPDE

In this tutorial, we show how to use the API SPDE.

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import numpy as np
import matplotlib.pyplot as plt

from sksparse.cholmod import cholesky
import scipy as sc
from scipy.sparse import *
from scipy.sparse.linalg import *
import numpy as np

In [ ]:
# Data
np.random.seed(123)
ndat = 1000

# Model

rangev = 0.2
sill = 1.
nugget = 0.1

# Grid 
nx = [50,50]
dx = [0.02,0.02]
x0 = [0,0]

#Grid meshing

nxm = [75,75]
dxm = [0.02,0.02]
x0m = [-0.25,-0.25]

### Grids definition

In [ ]:
grid = gl.DbGrid.create(nx,dx,x0)
gridExt = gl.DbGrid.create(nxm,dxm,x0m)
mesh = gl.MeshETurbo(gridExt)

### Model definition

In [ ]:
model = gl.Model.createFromParam(gl.ECov.BESSEL_K,param=1,range=rangev,sill=sill)
model.addCovFromParam(gl.ECov.NUGGET,sill=nugget)

### SPDE simulation

In [ ]:
spde = gl.SPDE()
spde.init(model,grid,None,gl.ESPDECalcMode.SIMUNONCOND)
spde.compute()

### Grid query

In [ ]:
iuid = spde.query(grid)

In [ ]:
ax = gp.grid(grid,title="Non Conditional Simulation")

### Data query

In [ ]:
dat = gl.Db.create()
dat["x"] = np.random.uniform(size=ndat)
dat["y"] = np.random.uniform(size=ndat)
dat.setLocators(["x","y"],gl.ELoc.X)
iuid = spde.query(dat)

### Kriging

In [ ]:
spdeRes = gl.SPDE()
spdeRes.init(model,grid,dat,gl.ESPDECalcMode.KRIGING,mesh)
spdeRes.compute()

In [ ]:
iuid = spdeRes.query(grid)
ax = gp.grid(grid, title="Estimation")

## Manually

### Aproj mesh to grid

In [ ]:
Pglg = gl.ProjMatrix(grid,mesh)
Apglg = Pglg.getAproj()
Atrg = gl.csToTriplet(Apglg)
Aprojg = sc.sparse.csc_matrix((np.array(Atrg.values), (np.array(Atrg.rows), np.array(Atrg.cols))),
                         shape=(Atrg.nrows,Atrg.ncols))

### Simulation

In [ ]:
Qtr = gl.csToTriplet(spdeRes.getPrecisionOp().getQ())
Q = sc.sparse.csc_matrix((np.array(Qtr.values), (np.array(Qtr.rows), np.array(Qtr.cols))))
cholQ =  cholesky(Q)


In [ ]:
u = np.random.normal(size = Q.shape[0])
gridExt["simuManual"] = cholQ.apply_Pt(cholQ.solve_Lt(1./np.sqrt(cholQ.D())*u))
gridExt.addSelection((gridExt["x1"]>0) & (gridExt["x2"]>0) & (gridExt["x1"]<1.) & (gridExt["x2"]<1.))
gp.grid(gridExt,"simuManual",usesel=False)
np.var(gridExt["simuManual"][np.where(gridExt["NewSel"]==1)])

### Kriging

In [ ]:
Pgl = gl.ProjMatrix(dat,mesh)
Apgl = Pgl.getAproj()
Atr = gl.csToTriplet(Apgl)
Aproj = sc.sparse.csc_matrix((np.array(Atr.values), (np.array(Atr.rows), np.array(Atr.cols))),
                         shape=(Atr.nrows,Atr.ncols))



Qop = Q + 1/nugget * Aproj.T @ Aproj
cholQop =  cholesky(Qop)

kriging = cholQop.solve_A(Aproj.T @ (dat["spde*"]/nugget))

In [ ]:
grid["manually"] = Aprojg @ kriging

In [ ]:
ax = plt.scatter(grid["manually"],grid["*kriging"],s=1)
plt.plot([-3,3],[-3,3],c="r")

## Likelihood

Manually with Cholesky vs. matrix-free approach with SPDE api.

In [ ]:
def solveMat(cholQop,x):
    return cholQop.solve_A(x)

def invSigma(sigma2,Aproj,cholQop,x):
    return 1./sigma2 * (x - 1./sigma2 * Aproj @ solveMat(cholQop, Aproj.T @ x))

def detQ(cholQ):
    return cholQ.logdet()

x = dat["spde.simu"]
ones = np.ones_like(x)
invSigmaOnes = invSigma(nugget,Aproj,cholQop,ones)
mu  = np.sum(x * invSigmaOnes)/np.sum( ones * invSigmaOnes) 
quad = np.sum((x-mu)*invSigma(nugget,Aproj,cholQop,x-mu))
logdet = len(x) * np.log(nugget) - detQ(cholQ) + detQ(cholQop)

print("logdet_chol",logdet)
print("quad_chol",quad)
print("like_chol",-0.5 * (quad + logdet))

In [ ]:
a = spdeRes.computeQuad()
print("relative difference quadratic",(a-quad)/quad)

In [ ]:
pcm = spdeRes.getPrecisionKriging()
a = detQ(cholQop)
b = pcm.computeLogDetOp(1,0)
print("log_det_op_chol",a)
print("log_det_op_api",b)
print("relative difference",(b-a)/a)

In [ ]:
a = detQ(cholQ)
b = pcm.computeLogDetQ(10,0)
print("log_det_Q_chol",a)
print("log_det_Q_api",b)
print("relative difference",(b-a)/a)

In [ ]:
a = -0.5 * (quad + logdet)
b = spdeRes.computeLogLike(100,0)
print("likelihood api",a)
print("likelihood_chol",b)
print((b-a)/a)